# QAOA Seeder for LABS problem

## Initial configuration

In this section we defined profiles of excecution, to optimize the control over the different tests and different platforms

In [2]:
import numpy as np
import pandas as pd
import time
from qaoa_quantum import qaoa_best_bitstring
import matplotlib.pyplot as plt
#install if needed
#!pip install pytest
import pytest

profiles = {
    "cpu_fast": dict(grid_points=5, shots_score=200, shots_final=300, top_k=2),
    "cpu_stronger": dict(grid_points=7, shots_score=150, shots_final=800, top_k=2),
    "gpu_basic": dict(grid_points=5, shots_score=200, shots_final=800, top_k=2, target="nvidia"),
    "gpu_premium": dict(grid_points=11, shots_score=250, shots_final=2000, top_k=3, target="nvidia"),
}


def plot_time_vs_N(df):
    for prof, g in df.groupby("profile"):
        g = g.sort_values("N")
        plt.plot(g["N"], g["time_sec"], marker="o", label=prof)
    plt.xlabel("N")
    plt.ylabel("Time (sec)")
    plt.title("QAOA runtime vs N")
    plt.grid(True)
    plt.legend()
    plt.show()

## qBraid testing

In this section, using only CPU hardware, we test diferent scenarios and compare the evolution over time

In [3]:
def run_one(N: int, profile_name: str, seed: int = 123, cost_scale: float = 2.0):
    cfg = profiles[profile_name]

    gammas = np.linspace(0.1, 1.0, cfg["grid_points"])
    betas  = np.linspace(0.1, 1.0, cfg["grid_points"])

    t0 = time.perf_counter()
    res = qaoa_best_bitstring(
        N=N,
        gammas=gammas,
        betas=betas,
        shots_score=cfg["shots_score"],
        shots_final=cfg["shots_final"],
        top_k=cfg["top_k"],
        target=cfg.get("target", None),
        seed=seed,
        cost_scale=cost_scale,
    )
    t1 = time.perf_counter()

    return {
        "profile": profile_name,
        "N": N,
        "grid_points": cfg["grid_points"],
        "shots_score": cfg["shots_score"],
        "shots_final": cfg["shots_final"],
        "top_k": cfg["top_k"],
        "target": cfg.get("target", None),
        "best_energy": res["best_energy"],
        "best_bits": res["best_bits"],
        "best_params": res["best_params"][:2],
        "time_sec": t1 - t0,
    }

def run_scenario(profile_name: str, Ns, seed: int = 123, cost_scale: float = 2.0):
    rows = []
    for N in Ns:
        rows.append(run_one(N, profile_name, seed=seed, cost_scale=cost_scale))
    return pd.DataFrame(rows)

Retunring the results from the scenarios, we plot the different result to see time saturation as expected

In [3]:
scenarios = {
    "S1_smoke": [7, 9, 11, 15, 20, 25],
    "S2_mid": [15, 20],
    "S3_high": [25, 30],
}

df_cpu = run_scenario("cpu_fast", scenarios["S1_smoke"])
df_cpu

,profile,N,grid_points,shots_score,shots_final,top_k,target,best_energy,best_bits,best_params,time_sec
0,cpu_fast,7,4,80,300,2,None,3,"[0, 0, 0, 1, 1, 0, 1]","[(3, 0.1, 0.7), (3, 0.4, 0.1)]",0.284982
1,cpu_fast,9,4,80,300,2,None,12,"[1, 0, 1, 1, 1, 1, 1, 0, 0]","[(12, 0.1, 0.1), (12, 0.1, 0.4)]",0.160346
2,cpu_fast,11,4,80,300,2,None,5,"[1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0]","[(5, 0.4, 0.1), (5, 0.4, 0.4)]",0.333396
3,cpu_fast,15,4,80,300,2,None,23,"[1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1]","[(23, 0.4, 0.1), (23, 1.0, 0.7)]",1.183934
4,cpu_fast,20,4,80,300,2,None,58,"[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, ...","[(46, 0.1, 1.0), (50, 1.0, 0.1)]",4.357736
5,cpu_fast,25,4,80,300,2,None,100,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ...","[(80, 0.7, 0.4), (88, 0.7, 0.7)]",47.563585


In [ ]:
plot_time_vs_N(df_cpu)

## Nvidia Brev Callibration

This section contains the 2 steep callibration strategy

### First steep

With a fixed N, we move grid points to find a good sweet point to work

### Second steep

The selected grid point is used to test saturation of N parameter

In [21]:
def calibrate_grid_points(
    profile_name: str,
    N_fixed: int,
    grid_candidates=(5, 7, 9),
    seed: int = 123,
    cost_scale: float = 2.0,
    metric: str = "energy_time",
    update_profile: bool = True,
):
    """
    Calibra grid_points manteniendo todo lo demás fijo (shots/top_k/target del profile).
    metric:
      - "best_energy"   (menor mejor)
      - "time_sec"      (menor mejor)
      - "energy_time"   (best_energy * time_sec, menor mejor)  recomendado
    update_profile:
      - si True, actualiza profiles[profile_name]["grid_points"] con el mejor gp
    """
    base_cfg = profiles[profile_name].copy()
    rows = []

    for gp in grid_candidates:
        # temporalmente “inyectar” grid_points sin cambiar el profile
        profiles[profile_name]["grid_points"] = gp
        rows.append(run_one(N_fixed, profile_name, seed=seed, cost_scale=cost_scale))

    # restaurar (por si update_profile=False)
    profiles[profile_name] = base_cfg

    df = pd.DataFrame(rows)
    df["energy_time"] = df["best_energy"] * df["time_sec"]

    if metric not in df.columns:
        raise ValueError(f"Metric '{metric}' not found. Use one of: best_energy, time_sec, energy_time.")

    df_sorted = df.sort_values(metric, ascending=True).reset_index(drop=True)
    best_gp = int(df_sorted.loc[0, "grid_points"])

    if update_profile:
        profiles[profile_name]["grid_points"] = best_gp

    return df_sorted, best_gp


def scale_after_calibration(
    profile_name: str,
    Ns,
    seed: int = 123,
    cost_scale: float = 2.0,
):
    """
    Corre el barrido de N usando el grid_points que está actualmente en profiles[profile_name].
    (Por ejemplo, luego de calibrate_grid_points(update_profile=True).)
    """
    return run_scenario(profile_name, Ns, seed=seed, cost_scale=cost_scale)


After first steep, best gird point is used to get the best parameters

In [22]:
df_calib, best_gp = calibrate_grid_points(
    profile_name="gpu_basic",
    N_fixed=20,
    grid_candidates=(5, 7, 9),
    metric="energy_time",     # recomendado
    update_profile=True       # deja guardado best_gp en profiles["gpu_basic"]["grid_points"]
)

df_calib, best_gp

(     profile   N  grid_points  shots_score  shots_final  top_k  target  \
 0  gpu_basic  20            5           80           30      2  nvidia   
 1  gpu_basic  20            7           80           30      2  nvidia   
 2  gpu_basic  20            9           80           30      2  nvidia   
 
    best_energy                                          best_bits  \
 0           70  [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, ...   
 1           98  [0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, ...   
 2           90  [0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, ...   
 
                                   best_params   time_sec  energy_time  
 0          [(54, 0.775, 1.0), (54, 1.0, 1.0)]   6.815876   477.111326  
 1          [(34, 0.85, 1.0), (42, 0.7, 0.85)]  12.918176  1265.981216  
 2  [(38, 0.55, 0.8875), (42, 0.8875, 0.6625)]  21.454336  1930.890263  ,
 5)

Simulations over N parameter brings saturation over bitstring length, compared vs time

In [5]:
Ns = [7, 9, 11, 15, 20, 25]
df_gpu = run_scenario("gpu_basic", Ns)
df_gpu

,profile,N,grid_points,shots_score,shots_final,top_k,target,best_energy,best_bits,best_params,time_sec
0,gpu_basic,7,5,200,800,2,nvidia,3,"[0, 0, 0, 1, 1, 0, 1]","[(3, 0.1, 0.1), (3, 0.1, 0.325)]",0.207047
1,gpu_basic,9,5,200,800,2,nvidia,12,"[1, 1, 0, 0, 1, 0, 1, 1, 1]","[(12, 0.1, 0.1), (12, 0.1, 0.325)]",0.341657
2,gpu_basic,11,5,200,800,2,nvidia,5,"[0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1]","[(5, 0.1, 0.55), (5, 0.325, 0.1)]",0.607245
3,gpu_basic,15,5,200,800,2,nvidia,23,"[0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0]","[(15, 0.55, 0.325), (15, 0.55, 0.775)]",1.937669
4,gpu_basic,20,5,200,800,2,nvidia,46,"[1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, ...","[(42, 0.1, 1.0), (46, 0.1, 0.775)]",6.832855
5,gpu_basic,25,5,200,800,2,nvidia,100,"[0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, ...","[(76, 0.325, 0.1), (88, 0.55, 1.0)]",71.492392


In [5]:
plot_time_vs_N(df_gpu)

NameError: name 'df_gpu' is not defined

## Unitary tests

Defined unitary test to ensure the sistem is running without problems, using small N known parameters

In [51]:
import sys
!{sys.executable} -m pytest -v tests/test_qaoa_quantum.py -s

============================= test session starts ==============================
platform linux -- Python 3.11.9, pytest-9.0.2, pluggy-1.5.0 -- /home/jovyan/.qbraid/environments/bewp/pyenv/bin/python
cachedir: .pytest_cache
rootdir: /home/jovyan/2026-NVIDIA/team-submissions
plugins: anyio-4.12.1
collected 2 items                                                              

tests/test_qaoa_quantum.py::test_reverse_symmetry_post_qaoa PASSED
tests/test_qaoa_quantum.py::test_expected_energy_from_table PASSED

============================== 2 passed in 3.33s ===============================


In [ ]:
# Benchmark CPU vs GPU: guarda tabla tiempo vs N + imprime comparativo + gráfica
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Asegúrate de tener definidos:
# - profiles (dict)
# - run_one(N, profile_name, ...)
# - (opcional) cudaq si vas a setear target en GPU
try:
    import cudaq
except Exception:
    cudaq = None


def _set_target_from_profile(profile_name: str):
    """Setea target una sola vez (si aplica)."""
    if cudaq is None:
        return
    tgt = profiles[profile_name].get("target", None)
    if tgt:
        cudaq.set_target(tgt)


def bench_avg_time(profile_name: str, N: int, reps: int = 3, warmup: bool = True,
                   seed: int = 123, cost_scale: float = 2.0):
    """
    Corre run_one varias veces y devuelve el promedio del time_sec.
    warmup=True ejecuta una corrida previa que NO se cuenta.
    """
    _set_target_from_profile(profile_name)

    if warmup:
        _ = run_one(N, profile_name, seed=seed, cost_scale=cost_scale)

    times = []
    last = None
    for _ in range(reps):
        out = run_one(N, profile_name, seed=seed, cost_scale=cost_scale)
        times.append(out["time_sec"])
        last = out
    return float(np.mean(times)), float(np.std(times)), last


def benchmark_cpu_vs_gpu(
    Ns,
    cpu_profile: str,
    gpu_profile: str,
    reps: int = 3,
    warmup: bool = True,
    seed: int = 123,
    cost_scale: float = 2.0,
):
    rows = []

    for N in Ns:
        cpu_mean, cpu_std, cpu_last = bench_avg_time(cpu_profile, N, reps=reps, warmup=warmup, seed=seed, cost_scale=cost_scale)
        gpu_mean, gpu_std, gpu_last = bench_avg_time(gpu_profile, N, reps=reps, warmup=warmup, seed=seed, cost_scale=cost_scale)

        rows.append({
            "N": N,
            "cpu_profile": cpu_profile,
            "gpu_profile": gpu_profile,

            "cpu_time_mean": cpu_mean,
            "cpu_time_std": cpu_std,
            "gpu_time_mean": gpu_mean,
            "gpu_time_std": gpu_std,

            "speedup_cpu_over_gpu": (cpu_mean / gpu_mean) if gpu_mean > 0 else np.nan,

            # opcional: energías (última corrida)
            "cpu_best_energy_last": cpu_last.get("best_energy", None) if cpu_last else None,
            "gpu_best_energy_last": gpu_last.get("best_energy", None) if gpu_last else None,

            # opcional: parámetros y bits (última corrida)
            "cpu_best_params_last": cpu_last.get("best_params", None) if cpu_last else None,
            "gpu_best_params_last": gpu_last.get("best_params", None) if gpu_last else None,
        })

    df = pd.DataFrame(rows).sort_values("N").reset_index(drop=True)

    # Tabla comparativa “bonita” para imprimir
    df_print = df[[
        "N",
        "cpu_time_mean", "gpu_time_mean",
        "speedup_cpu_over_gpu",
        "cpu_best_energy_last", "gpu_best_energy_last",
    ]].copy()

    return df, df_print


def plot_cpu_vs_gpu_times(df, title="CPU vs GPU runtime vs N"):
    plt.figure()
    plt.plot(df["N"], df["cpu_time_mean"], marker="o", label="CPU")
    plt.plot(df["N"], df["gpu_time_mean"], marker="o", label="GPU")
    plt.xlabel("N")
    plt.ylabel("Mean time (sec)")
    plt.title(title)
    plt.grid(True)
    plt.legend()
    plt.show()


# -------------------------
# EJEMPLO DE USO
# -------------------------
# Define N list (ajusta a tu presupuesto)
Ns = [7, 9, 11, 15, 20]

# Escoge perfiles a comparar (pueden ser los tuyos existentes)
cpu_profile = "cpu_fast"
gpu_profile = "gpu_basic"

df_full, df_comp = benchmark_cpu_vs_gpu(
    Ns=Ns,
    cpu_profile=cpu_profile,
    gpu_profile=gpu_profile,
    reps=3,        # sube a 5 si quieres menos ruido
    warmup=True,   # recomendado
    seed=123,
    cost_scale=2.0
)

print("Comparative table (mean times, speedup, last energies):")
display(df_comp)

plot_cpu_vs_gpu_times(df_full, title=f"CPU({cpu_profile}) vs GPU({gpu_profile}) runtime vs N")
